In [1]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
import numpy as np
import itertools
from sys import stdout


### Creates a coarse-grained (CG) element to be used in simulations.

This is an interesting feature that you'd only encounter in coarse-grained simulations. In CG, of course, you'd NEED to deifne the "atomic number", mass, charge etc of the CG center that you are creating. That's what we do next.

In [2]:
cgElement = app.Element(number=1000, name='CG-element', symbol='CG', mass=120)

'''
#Given a set of masses, create a set of elements
if 'elements_initialized' not in globals():
    elements = []
    for i, mass in enumerate(MASS):
        element = app.Element(number=1000 + i, name=f'C{i+1}', symbol=f'C{i+1}', mass=mass)
        elements.append(element)
    elements_initialized = True
'''

"\n#Given a set of masses, create a set of elements\nif 'elements_initialized' not in globals():\n    elements = []\n    for i, mass in enumerate(MASS):\n        element = app.Element(number=1000 + i, name=f'C{i+1}', symbol=f'C{i+1}', mass=mass)\n        elements.append(element)\n    elements_initialized = True\n"

In [3]:
# Make an empty topology
topology = app.Topology()

### The System

In this notebook, we will create a simulation box with 100 polymer chains, with 10 beads each. 

In [4]:
# Number of polymer chains
M = 100
# Number of atoms in each chain
N = 10

# Add each chain to the topology
for m in range(M):
    chain = topology.addChain()
    atom1 = topology.addAtom(name="CG-bead", element=cgElement, residue=topology.addResidue(name="CG-residue", chain=chain))
    
    for i in range(1, N):
        atom2 = topology.addAtom(name="CG-bead", element=cgElement, residue=topology.addResidue(name="CG-residue", chain=chain))
        topology.addBond(atom1, atom2)
        atom1 = atom2

# Check the topology
print(topology)


<Topology; 100 chains, 1000 residues, 1000 atoms, 900 bonds>


Now, we place the beads in a grid. Distance between beads in a single polymer is 0.38 nm

In [5]:
# Initialize an empty list to store positions
positions = []
# Loop over each polymer chain
for m in range(M):
    # Calculate the initial position for the first bead in the chain
    x0 = np.array(((m % 10) * 1.0, (m // 10) * 1.0, 0))
    positions.append(x0)    
    # Loop over the remaining beads in the chain
    for i in range(1, N):
        # Calculate the position for the next bead in the chain
        xi = positions[-1] + np.array((0, 0, 0.38))
        positions.append(xi)

# Convert the list of positions into an OpenMM Quantity with units of nanometers
positions = positions * unit.nanometer
# Ensure the number of positions matches the number of atoms in the topology
assert len(positions) == topology.getNumAtoms()
# Set the periodic box vectors to create a cubic box with a length of 11 nm
topology.setPeriodicBoxVectors(np.eye(3) * 11.0 * unit.nanometers)


In [6]:
# output the initial configuration. Save it to a file
with open('initial_config.pdb','w') as f:
    app.PDBFile.writeFile(topology, positions, f)


<center><img src="initial_state.png" alt="Initial State" width="400"/></center>

At this point, we now move to OpenMM, and prepare the simulation.

In [7]:
# create the system and add the particles to it
system = mm.System()
system.setDefaultPeriodicBoxVectors(*topology.getPeriodicBoxVectors())
for atom in topology.atoms():
    system.addParticle(atom.element.mass)

### Defining the Force Field

This section summarizes the custom force field parameters used in the simulation. The interactions include bonded (bonds, angles) and non-bonded (van der Waals) terms.

---

#### **Bonds**

Modeled as harmonic springs:

$$
U_{\text{bond}}(r) = \frac{1}{2} k_b (r - r_0)^2
$$

**Parameters**:
- $r_0 = 0.38 \, \text{nm}$ (equilibrium bond length)  
- $k_b = 1000 \, \text{kJ/mol/nm}^2$ (bond spring constant)

---

#### **Angles**

Modeled using a cosine-based harmonic potential:

$$
U_{\text{angle}}(\theta) = \frac{1}{2} k_a \left( \cos\theta - \cos\theta_0 \right)^2
$$

**Parameters**:
- $\theta_0 = 180^\circ$ (equilibrium angle)  
- $k_a = 10 \, \text{kJ/mol}$

> This form avoids singularities near \(180^\circ\) and is numerically stable.

---

#### **Non-Bonded Interactions**

Modeled using the Lennard-Jones (LJ) potential:

$$
U_{\text{LJ}}(r) = 4\epsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^6 \right]
$$

**Parameters**:
- $\sigma = 0.5 \, \text{nm}$ (particle diameter)  
- $\epsilon = 1 \, \text{kJ/mol}$ (interaction strength)

In [8]:
harmonic_bond_force = mm.HarmonicBondForce()

# Add each bond to the force from the topology
for bond in topology.bonds():
    harmonic_bond_force.addBond(bond.atom1.index, bond.atom2.index, 0.38, 1000)

'''
custom_angle = mm.CustomAngleForce("0.5*k*(cos(theta)-cos(theta0))^2")
custom_angle.addPerAngleParameter('k')
custom_angle.addPerAngleParameter('theta0')
# Loop through all chains and assign angles for each three bonded atoms
for chain in topology.chains():
    atoms = list(chain.atoms())
    for i in range(len(atoms) - 2):
        custom_angle.addAngle(atoms[i].index, atoms[i+1].index, atoms[i+2].index, [10, 3.14159])
'''
        
# Define a Lennard-Jones potential
expression = '4*epsilon*((sigma/r)^12-(sigma/r)^6);'\
            + ' sigma=0.5*(sigma1+sigma2);'\
            + ' epsilon=sqrt(epsilon1*epsilon2)'

custom_nb_force = mm.CustomNonbondedForce(expression)

custom_nb_force.addPerParticleParameter('sigma')
custom_nb_force.addPerParticleParameter('epsilon')

# Add the parameters for each particle
for atom in topology.atoms():
    custom_nb_force.addParticle([0.5, 1.0])

# Create exclusions for directly bonded atoms
custom_nb_force.createExclusionsFromBonds([(bond[0].index, bond[1].index) for bond in topology.bonds()], 1)

# Set a cutoff of 1.5nm
custom_nb_force.setNonbondedMethod(mm.CustomNonbondedForce.CutoffPeriodic)
custom_nb_force.setCutoffDistance(1.5*unit.nanometers)

# Add the forces to the system
system.addForce(harmonic_bond_force)
#system.addForce(custom_angle)
system.addForce(custom_nb_force)

1

In [9]:
with open('system1.xml', 'w') as output:
    output.write(mm.XmlSerializer.serialize(system))

In [10]:
#Running the simulation - very similar to proein-in_water simulation
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 0.01/unit.picosecond, 0.010*unit.picoseconds)
simulation = app.Simulation(topology, system, integrator)
simulation.context.setPositions(positions)

# setup simulation reporters
# Write the trajectory to a file called 'traj.dcd'
simulation.reporters.append(app.DCDReporter('traj.dcd', 1000, enforcePeriodicBox=True))

# Report information to the screen as the simulation runs
simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True, speed=True))


# NVT equilibration
simulation.step(10000)
# Add a barostat
barostatIndex=system.addForce(mm.MonteCarloBarostat(1.0*unit.bar, 300*unit.kelvin))
simulation.context.reinitialize(preserveState=True)
# Run NPT equilibration
simulation.step(100000)


# output the equilibrated configuration
with open('equilibrated_config.pdb','w') as f:
    state = simulation.context.getState(getPositions=True, enforcePeriodicBox=True)
    topology.setPeriodicBoxVectors(state.getPeriodicBoxVectors())
    app.PDBFile.writeFile(topology, state.getPositions(), f)


#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Speed (ns/day)"
1000,-2453.382788192481,190.58470681545617,1331.0,0
2000,-2720.090639166534,227.2137301670289,1331.0,7.76e+03
3000,-2625.9057470560074,230.86098169369495,1331.0,7.69e+03
4000,-2626.5881546735764,241.5779562315645,1331.0,7.73e+03
5000,-2577.7014751434326,251.7451373448813,1331.0,7.75e+03
6000,-2517.820065975189,254.74070172210716,1331.0,7.77e+03
7000,-2414.6818838119507,256.26347188495043,1331.0,7.78e+03
8000,-2293.7195863723755,256.2833929320893,1331.0,7.78e+03
9000,-2188.785840988159,258.75634833107773,1331.0,7.81e+03
10000,-2205.5437321662903,274.0066829934392,1331.0,7.81e+03
11000,-2143.363098859787,276.73157162848986,1277.2620912547507,6.81e+03
12000,-2218.1778616905212,288.52972561459904,1206.2374862301833,6.1e+03
13000,-2166.920965194702,285.4258446444209,1224.558073190987,5.69e+03
14000,-2204.0869675278664,284.7227738156243,1093.843984476562,5.22e+03
15000,-2159.6247301101685,285.86997983

<center><img src="final_state.png" alt="Final State" width="400"/></center>